# 🚆Train_cost
지하철 비용 추산에 사용될 데이터 가공

### TODO
- [ ] 데이터 불러오기
- [ ] 서울에 지나는 역들로만 추리기
- [ ] - 1250 처리

### 0. import

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import set_matplotlib_formats
import koreanize_matplotlib
import folium
from folium.plugins import MarkerCluster
import json

### 1. 데이터 불러오기

In [3]:
cost = pd.read_csv("./train_cost.csv", encoding="cp949");
cost

,연번,출발역번호,출발역명,도착역번호,도착역명,거리,수도권내,수도권외,공항철도,공항철도2,운임,정기권단계,서울전용여부
0,1,150,서울역,150,서울역,0.0,0.0,0.0,0.0,0.0,1250,1,YES
1,2,150,서울역,151,시청,1.1,1.1,0.0,0.0,0.0,1250,1,YES
2,3,150,서울역,152,종각,2.1,2.1,0.0,0.0,0.0,1250,1,YES
3,4,150,서울역,153,종로3가,2.9,2.9,0.0,0.0,0.0,1250,1,YES
4,5,150,서울역,154,종로5가,3.8,3.8,0.0,0.0,0.0,1250,1,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219550,219551,2828,남위례,4925,걸포북변,49.6,49.6,0.0,0.0,0.0,2050,7,NO
219551,219552,2828,남위례,4926,사우(김포시청),47.8,47.8,0.0,0.0,0.0,2050,7,NO
219552,219553,2828,남위례,4927,풍무,46.4,46.4,0.0,0.0,0.0,2050,7,NO
219553,219554,2828,남위례,4928,고촌,42.8,42.8,0.0,0.0,0.0,1950,6,NO


In [4]:
cost = cost.drop(columns=["연번", "거리", "수도권내", "수도권외", "공항철도", "공항철도2", "정기권단계"])
cost

,출발역번호,출발역명,도착역번호,도착역명,운임,서울전용여부
0,150,서울역,150,서울역,1250,YES
1,150,서울역,151,시청,1250,YES
2,150,서울역,152,종각,1250,YES
3,150,서울역,153,종로3가,1250,YES
4,150,서울역,154,종로5가,1250,YES
...,...,...,...,...,...,...
219550,2828,남위례,4925,걸포북변,2050,NO
219551,2828,남위례,4926,사우(김포시청),2050,NO
219552,2828,남위례,4927,풍무,2050,NO
219553,2828,남위례,4928,고촌,1950,NO


In [6]:
train_seoul_station = pd.read_csv("../hub/result_all_train_station_in_seoul.csv", encoding="cp949");
train_seoul_station

,route_name,주소,stat_id,stat_name,lat,lng,geohash
0,공항철도,서울특별시 용산구 청파로 378,4001.0,서울역,37.553247,126.969769,wydm9k
1,공항철도,서울특별시 마포구 마포대로 92,4002.0,공덕,37.542530,126.952024,wydm8f
2,공항철도,서울특별시 마포구 양화로 188,4003.0,홍대입구,37.557438,126.926715,wydm8m
3,공항철도,서울특별시 마포구 성암로 184,4004.0,디지털미디어시티,37.576958,126.898609,wydjz8
4,공항철도,서울특별시 강서구 마곡중앙로 지하 135,4020.0,마곡나루,37.565543,126.827378,wydjwn
...,...,...,...,...,...,...,...
445,수도권 8호선,경기도 성남시 수정구 산성대로 지하200(수진동),825.0,수진,37.437428,127.140722,wydkvp
446,수도권 8호선,경기도 성남시 수정구 산성대로 지하100(수진동),826.0,모란,37.433824,127.129837,wydkuy
447,수인분당선,경기도 성남시 중원구 성남대로 지하 1147(성남동 1575),1525.0,모란,37.432052,127.129104,wydkuy
448,수인분당선,경기도 성남시 수정구 성남대로 지하 1229번지,1524.0,태평,37.440019,127.127709,wydkuz


In [11]:
cost = cost.astype({"출발역번호": int, "도착역번호": int})
train_seoul_station = train_seoul_station.astype({"stat_id": int})

In [17]:
cost_station_name = set(cost["출발역명"]) | set(cost["도착역명"])
seoul_station_name = set(train_seoul_station["stat_name"])

print(seoul_station_name  - cost_station_name)

{'월곡', '한성대입구', '강변', '남한산성입구', '신정', '상봉', '하남시청', '성신여대입구', '충정로', '회현', '온수', '공릉', '용두', '월드컵경기장', '4.19 민주묘지', '교대', '흑석', '숭실대입구', '왕십리', '고려대', '광흥창', '대림', '올림픽공원', '증산', '숙대입구', '서울대입구', '광화문', '화랑대', '안암', '새절', '청량리', '미아', '남부터미널', '대흥', '이촌', '양재', '삼성', '수유', '양재시민의숲', '경복궁', '녹사평', '봉화산', '굽은다리', '동작', '잠실', '구의', '오목교', '상월곡'}


In [19]:
cost = cost.replace({
    "월곡(동덕여대)": "월곡",
    "한성대입구(삼선교)": "한성대입구",
    "강변(동서울터미널)": "강변",
    "남한산성입구(성남법원.검찰청)": "남한산성입구",
    "신정(은행정)": "신정",
    "상봉(시외버스터미널)": "상봉",
    "하남시청(덕풍·신장)": "하남시청",
    "성신여대입구(돈암)": "성신여대입구",
    "충정로(경기대입구)": "충정로",
    "회현(남대문시장)": "회현",
    "온수(성공회대입구)": "온수",
    "공릉(서울과학기술대)": "공릉",
    "용두(동대문구청)": "용두",
    "월드컵경기장(성산)": "월드컵경기장",
    "4.19민주묘지": "4.19 민주묘지",
    "교대(법원.검찰청)": "교대",
    "흑석(중앙대입구)": "흑석",
    "숭실대입구(살피재)": "숭실대입구",
    "왕십리(성동구청)": "왕십리",
    "고려대(종암)": "고려대",
    "광흥창(서강)": "광흥창",
    "대림(구로구청)": "대림",
    "올림픽공원(한국체대)": "올림픽공원",
    "증산(명지대앞)": "증산",
    "숙대입구(갈월)": "숙대입구",
    "서울대입구(관악구청)": "서울대입구",
    "광화문(세종문화회관)": "광화문",
    "화랑대(서울여대입구)": "화랑대",
    "안암(고대병원앞)": "안암",
    "새절(신사)": "새절",
    "청량리(서울시립대입구)": "청량리",
    "미아(서울사이버대학)": "미아",
    "남부터미널(예술의전당)": "남부터미널",
    "대흥(서강대앞)": "대흥",
    "이촌(국립중앙박물관)": "이촌",
    "양재(서초구청)": "양재",
    "삼성(무역센터)": "삼성",
    "수유(강북구청)": "수유",
    "양재시민의숲(매헌)": "양재시민의숲",
    "경복궁(정부서울청사)": "경복궁",
    "녹사평(용산구청)": "녹사평",
    "봉화산(서울의료원)": "봉화산",
    "굽은다리(강동구민회관앞)": "굽은다리",
    "동작(현충원)": "동작",
    "잠실(송파구청)": "잠실",
    "구의(광진구청)": "구의",
    "오목교(목동운동장앞)": "오목교",
    "상월곡(한국과학기술연구원)": "상월곡"
})


In [20]:
cost_station_name = set(cost["출발역명"]) | set(cost["도착역명"])
seoul_station_name = set(train_seoul_station["stat_name"])

print(seoul_station_name - cost_station_name)

set()


In [24]:
dropIndex = [];

for i in range(len(cost)):
    row = cost.loc[i]
    
    if (row["출발역명"] not in seoul_station_name) or (row["도착역명"] not in seoul_station_name) :
        dropIndex.append(i)
        
cost = cost.drop(dropIndex)

In [26]:
cost = cost.reset_index(drop=True)
cost

,출발역번호,출발역명,도착역번호,도착역명,운임,서울전용여부
0,150,서울역,150,서울역,1250,YES
1,150,서울역,151,시청,1250,YES
2,150,서울역,152,종각,1250,YES
3,150,서울역,153,종로3가,1250,YES
4,150,서울역,154,종로5가,1250,YES
...,...,...,...,...,...,...
123470,2828,남위례,4710,정릉,1550,YES
123471,2828,남위례,4711,성신여대입구,1550,YES
123472,2828,남위례,4712,보문,1450,YES
123473,2828,남위례,4713,신설동,1450,YES


In [27]:
cost.to_csv("./result_train_cost.csv")